In [1]:
from __future__ import print_function

import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten, GlobalMaxPooling2D, Lambda, UpSampling2D, BatchNormalization
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.preprocessing.image import load_img

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
from tqdm import tqdm
import os
import csv
import os
import pickle
import pandas as pd

%matplotlib inline

In [2]:
new_size = 260

folder = 'test_set'
images_folder = os.path.join(folder, 'output')
labels_file = os.path.join(folder, 'out.csv')
classes_file = os.path.join('CUB_200_2011', 'classes.txt')
df = pd.read_csv(labels_file)

In [3]:
classes = dict()

with open(classes_file, 'r') as f:
    data = f.read()
data = data.strip().split('\n')

for line in data:
    num, label = line.split(' ')
    num = int(num)
    classes[label] = num

In [4]:
images = df['name'].values
labels = df['label'].values
labels = [classes[label]-1 for label in labels]

In [5]:
x_test = np.zeros((len(images), new_size, new_size, 3),np.float32)
y_test = np.zeros(len(images), np.float32)

for i, image in enumerate(tqdm(images)):
    x = Image.open(os.path.join(images_folder, image)).convert('RGB').resize((new_size, new_size), Image.ANTIALIAS)
    x = np.array(x).astype('float32')
    x_test[i,:,:,:] = x
    
    y_test[i] = labels[i]

x_test = x_test/255
y_test = tf.keras.utils.to_categorical(y_test)

100%|██████████| 1674/1674 [00:05<00:00, 287.22it/s]


In [6]:
name = 'efficientNetB2_batch_8.h5'
model = tf.keras.models.load_model(name)

In [7]:
y_pred = model.predict(x_test)
result = np.sum(np.argmax(y_test, axis=1) == np.argmax(y_pred, axis=1))/len(y_pred)
print(result)

0.6917562724014337


In [15]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name):
    with open(name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [26]:
data = load_obj('history')

In [27]:
data.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy', 'lr'])

In [10]:
preds = np.argmax(y_pred, axis=1)
labels = np.argmax(y_test, axis=1)
acc = result

In [13]:
out = dict()
out['acc'] = acc
out['preds'] = preds
out['labels'] = labels

save_obj(out, 'efficientNet')

In [12]:
def save_obj(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)